In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import vtk
import math
from vtk.util import numpy_support

from params import *



In [28]:
def vti_reader(file_path):
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(file_path)
    reader.Update()
    data = reader.GetOutput()
    
    # vtk res
    dataset = pd.DataFrame(np.array([numpy_support.vtk_to_numpy(data.GetPointData().GetArray(i)) for i in range(data.GetPointData().GetNumberOfArrays())])).transpose()
    
    dataset.columns=[data.GetPointData().GetArrayName(i) for i in range(data.GetPointData().GetNumberOfArrays())]
    
    # vtk coord
    dimensions = data.GetDimensions()
    spacing=data.GetSpacing()
    origin=data.GetOrigin()
    
    coordinates=[]
    
    for k in range(dimensions[2]):
        for j in range(dimensions[1]):
            for i in range(dimensions[0]):
                x = origin[0] + i * spacing[0]
                y = origin[1] + j * spacing[1]
                z = origin[2] + k * spacing[2]
                coordinates.append((x,y,z))
    dt_coordinates = pd.DataFrame(coordinates, columns=['x','y','z'])
    
    # result coord merge
    return pd.merge(dt_coordinates, dataset, left_index=True, right_index=True)

In [29]:
path_saved = 'D:\\coating_modulus\\outputs\\'
doe_name = 'twophase_coat_v7_2'

In [30]:
dirnames = os.listdir(path_saved+doe_name)
# 'windows'를 포함하는 이름만 선택
dir_windows = [name for name in dirnames if 'window' in name.lower()]
dir_windows.sort()
dir_windows

['window_0000',
 'window_0001',
 'window_0002',
 'window_0003',
 'window_0004',
 'window_0005',
 'window_0006',
 'window_0007',
 'window_0008',
 'window_0009',
 'window_0010',
 'window_0011']

In [33]:
file_path = path_saved + doe_name + '\\' + dir_windows[0] + '\\inferencers\\window_000t.vti'
vti_data = vti_reader(file_path)

c:\ProgramData\anaconda3\Lib\site-packages\vtkmodules\util\numpy_support.py:74: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  _vtk_np = {vtkConstants.VTK_BIT:numpy.bool,


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [36]:
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(file_path)
    reader.Update()
    data = reader.GetOutput()
    
    # vtk res
    #dataset = pd.DataFrame(np.array([numpy_support.vtk_to_numpy(data.GetPointData().GetArray(i)) for i in range(data.GetPointData().GetNumberOfArrays())])).transpose()


In [38]:
numpy_support.vtk_to_numpy(data.GetPointData().GetArray(0))

c:\ProgramData\anaconda3\Lib\site-packages\vtkmodules\util\numpy_support.py:74: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  _vtk_np = {vtkConstants.VTK_BIT:numpy.bool,


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations